In [ ]:
# Import Library
# --------------
import cv2
import os
import errno
import time
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from skimage import io, transform 

from tensorflow import keras

from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dense
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import metrics

from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score 

In [ ]:
# Set Directory
# -------------
BASE_DIR = 'D:/MDP/Skripsi/Program/Fingers-Dataset/Pre-Processing/'
TRAIN_DIR = os.path.join(BASE_DIR, (os.listdir(BASE_DIR)[1]+'/'))
VAL_DIR = os.path.join(BASE_DIR, (os.listdir(BASE_DIR)[2]+'/'))

# Check Image Shape Data Training
# -------------------------------
kelas = os.listdir(TRAIN_DIR)[0]    # [0] = 1L   
fpath = os.path.join(TRAIN_DIR, (kelas+'/'))  
fname = os.listdir(fpath)[0]        # File Name 
fdir = fpath+fname                  # File Directory

shape_old = cv2.imread(fdir).shape      # Shape (tuple)
x = list(shape_old)     
x[0] = x[1] = 227                       # Reshape (227,227,3)
input_shape = tuple(x)                  # Input Shape

print('File Directory :', fdir)
print('File Name      :', fname)
print('Image Shape    :', shape_old, '==> Original')
print('Image Reshape  :', input_shape, '==> AlexNet', '\n')

target_size = x = list(input_shape)
target_size.remove(3)
target_size = tuple(target_size)        # Target Size
print('Target Size    :', target_size)
print('Kelas          :', kelas)

io.imshow(transform.resize(io.imread(fdir), (x[0], x[1])))
io.show()

In [ ]:
# Config Arsitektur AlexNet CNN
# -----------------------------
model = Sequential()

# Layer Feature Learning
# ----------------------
model.add(Conv2D(16, kernel_size=11, strides=4, activation='relu', input_shape=input_shape))
model.add(AveragePooling2D(pool_size=3, strides=2))
# model.add(MaxPooling2D(pool_size=3, strides=2))

model.add(Conv2D(32, kernel_size=5, strides=1, activation='relu'))
model.add(AveragePooling2D(pool_size=3, strides=2))
# model.add(MaxPooling2D(pool_size=3, strides=2))

model.add(Conv2D(64, kernel_size=3, strides=1, activation='relu')) 
model.add(Conv2D(64, kernel_size=3, strides=1, activation='relu')) 
model.add(Conv2D(32, kernel_size=3, strides=1, activation='relu')) 
model.add(AveragePooling2D(pool_size=3, strides=2))
# model.add(MaxPooling2D(pool_size=3, strides=2))

# Layer Classification
# --------------------
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(12, activation='softmax')) # Output Class


# Compile Optimizer
# -----------------
# SGD, RMSprop, Adam
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.00001), #1.0e-5
              metrics=['acc'])

# Print Model AlexNet
model.summary()

In [ ]:
# Augmentasi Data Training & Validation
# -------------------------------------
batch_size_train = 300
batch_size_val = 120

Train_DataGen = ImageDataGenerator().flow_from_directory(
    TRAIN_DIR,
    target_size=target_size,
    batch_size=batch_size_train,
    class_mode='categorical',
    # shuffle=True
)

Val_DataGen = ImageDataGenerator().flow_from_directory(
    VAL_DIR,
    target_size=target_size,
    batch_size=batch_size_val,
    class_mode='categorical'
)

In [ ]:
# Train Model
# -----------

# AveragePooling, MaxPooling
# SGD, RMSprop, Adam

pooling_layer = 'AveragePooling'
optimizer = 'SGD'
#-------------------------------
model_dir = 'D:/MDP/Skripsi/Program/Python/AlexNet/Model/'+pooling_layer+'/'+optimizer+'/'
model_name = 'AlexNet-'+pooling_layer+'-'+optimizer
file_path = model_dir+model_name+'-{epoch:02d}-{acc:.4f}-{val_acc:.4f}.h5'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=file_path,
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True
)
s = time.time()
model_history = model.fit(
    Train_DataGen,
    steps_per_epoch=54,    # 16.200 images = batch_size * steps_per_epoch  => Train
    epochs=25,
    validation_data=Val_DataGen,
    validation_steps=27,    # 3.240 images = batch_size * steps_per_epoch   => Validation
    callbacks=[model_checkpoint_callback]
)
print('\n\nTotal time :', round(time.time() - s, 2), 'seconds')

In [ ]:
# Plot Training and Validation
# ----------------------------
plot_path = 'D:/MDP/Skripsi/Program/Python/AlexNet/Plot/'

try:
    os.mkdir(plot_path)
except OSError as exc:
    if exc.errno == errno.EEXIST and os.path.isdir(plot_path):
        pass
    else:
        raise

plot_name = model_name
plot_dir = plot_path+plot_name+'-Plot.png'

acc = model_history.history['acc']
val_acc = model_history.history['val_acc']
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

# Get Number Epoch
epochs = range(len(acc))

plt.figure(figsize=(12,6))
# Plot Training & Validation Acc Per Epoch
plt.subplot(1,2,1)
plt.xlabel('Epoch')
plt.plot(epochs,acc,'r',label='training')
plt.plot(epochs,val_acc,'b',label='validation')
plt.legend()
plt.title('Accuracy')

# Plot Training & Validation Loss Per Epoch
plt.subplot(1,2,2)
plt.xlabel('Epoch')
plt.plot(epochs,loss,'r',label='training')
plt.plot(epochs,val_loss,'b',label='validation')
plt.legend()
plt.title('Loss')

plt.suptitle(plot_name)
plt.tight_layout()
plt.savefig(plot_dir, facecolor='w')
plt.show()

In [ ]:
# Load Model
# ----------

# AveragePooling, MaxPooling
# SGD, RMSprop, Adam

pooling_layer = 'MaxPooling'
optimizer = 'Adam'
#-------------------------------
model_dir = 'D:/MDP/Skripsi/Program/Python/LeNet/Model/'+pooling_layer+'/'+optimizer+'/'

file_name = os.listdir(model_dir)[-1]  # index terakhir

model = load_model(os.path.join(model_dir, file_name))
model.summary()


# Set Folder Path And Label
# -------------------------

# Label True Classes
finger_list = ['0L','0R','1L','1R','2L','2R','3L','3R','4L','4R','5L','5R']
list_acc = []   
lbl_predict = []        # Label Predict
target_size = (227,227)
# batch_size = 20
data_class = 12         # Total Data Class 
data_per_label = 180    # Total Data Per Label ['0L','0R','1L','1R','2L','2R','3L','3R','4L','4R','5L','5R']
data_test = 2160        # Total Data Test

# Label Actual
lbl_actual = []
for i in range(data_class):
    for j in range(data_per_label):
        lbl_actual.append(i)
# print(lbl_actual)

# Label Finger Path
path = 'D:/MDP/Skripsi/Program/Fingers-Dataset/Pre-Processing/Testing/'

fingers_path = []

for i in range(len(finger_list)):
    fingers_path.append(path+finger_list[i])
# print(fingers_path)


# Count Accuracy Per Label
# ------------------------
for i in range(len(fingers_path)):
    fingers_img = []
    for j in os.listdir(fingers_path[i]):

        j = os.path.join(fingers_path[i], j)
        j = image.load_img(j, target_size=target_size)
        j = image.img_to_array(j)
        j = np.expand_dims(j, axis=0)
        fingers_img.append(j)

    # Set Stack up fingers_img for Prediction
    fingers_img = np.vstack(fingers_img)
    prediction_classes = np.argmax(model.predict(fingers_img), axis=-1)
    lbl_predict.extend(prediction_classes)

  # Print Result
    print('\n\n----------------------------------------------------------------------------')
    print('Label Name [', finger_list[i], '] == ', i+1, 
        '\nPrediction_Class = \n\n', prediction_classes+1, 
        '\nTotal True Prediction = ', list(prediction_classes).count(i),
        '\nTotal Data Test       = ', data_per_label,
        '\nAccuracy per Label    = ', round(int(list(prediction_classes).count(i)) / data_per_label * 100, 2),'%',
        '\n----------------------------------------------------------------------------\n')
    list_acc.append(list(prediction_classes).count(i))

print('Result Test Accuracy = ', '%.2f' % round((sum(list_acc)/data_test)*100, 2),'%','\n\n')


# Confusion Matrix
# ----------------
actual = lbl_actual
predicted = lbl_predict

print('---------------- Confusion Matrix ----------------\n\n','X = Predict\n Y = Actual\n\n', confusion_matrix(actual, predicted), '\n\n')
print(classification_report(actual, predicted, target_names=finger_list, digits=4))

print('\n------------------- Overall Score -------------------\n')
precision   = precision_score(actual, predicted, average='weighted') * 100
recall      = recall_score(actual, predicted, average='weighted') * 100
accuracy    = accuracy_score(actual, predicted) * 100
f1          = f1_score(actual, predicted, average='weighted') * 100

print('Precision    :', round(precision, 2), '%\n----------------------')
print('Recall       :', round(recall, 2), '%\n----------------------')
print('F1-Score     :', round(f1, 2), '%\n----------------------')
print('Accuracy     :', round(accuracy, 2), '%')



In [ ]:
# Prediksi 1 Gambar

kelas = os.listdir(TRAIN_DIR)[0]    # [0] = 1L   
fpath = os.path.join(TRAIN_DIR, (kelas+'/'))  
fname = os.listdir(fpath)[0]        # File Name 
fdir = fpath+fname                  # File Directory

img = image.load_img(fdir, target_size=target_size)
implot = plt.imshow(img)

im = image.img_to_array(img)
im = np.expand_dims(im, axis=0)

imx = np.vstack([im])
# prediction = model.predict_classes(imx, batch_size=1)
prediction = np.argmax(model.predict(imx), axis=-1)
prediction = prediction+1

print('\n')
print('File Name  :', fname)
print('Prediction :', prediction)
if (prediction==1):
    print('Prediction : 0L')
elif (prediction==2):
    print('Prediction : 0R')
elif (prediction==3):
    print('Prediction : 1L')    
elif (prediction==4):
    print('Prediction : 1R')    
elif (prediction==5):
    print('Prediction : 2L')    
elif (prediction==6):
    print('Prediction : 2R')
elif (prediction==7):
    print('Prediction : 3L')    
elif (prediction==8):
    print('Prediction : 3R')    
elif (prediction==9):
    print('Prediction : 4L')    
elif (prediction==10):
    print('Prediction : 4R')
elif (prediction==11):
    print('Prediction : 5L')    
elif (prediction==12):
    print('Prediction : 5R')
else:
    print('Result : Unknown')  

In [ ]:
import pandas as pd
from pandas import ExcelWriter

y_actual = pd.Series(actual, name='Actual')
y_pred = pd.Series(predicted, name='Predicted')

df_confusion = pd.crosstab(y_actual, y_pred)

print(df_confusion)

# df_confusion.to_csv('test.csv')
# df_confusion.to_excel('test.xlsx')
# # wr = ExcelWriter('test.xlsx')
# # df_confusion.to_excel(wr, 'Sheet1')
# # wr.save()

In [ ]:
class_accurracies = []
for lbl_actual in np.unique(y_actual):
    x = np.mean(y_pred[y_actual == lbl_actual] == lbl_actual)
    class_acc = round((x * 100),2)
    class_accurracies.append(class_acc)
    
print (class_accurracies)